In [1]:
import numpy as np
import pandas as pd
import glob
import os
import shutil

In [2]:
def parse_metainfo(root_path):
    train_list = pd.read_csv(os.path.join(root_path, 'train_list_rawframes.txt'), header=None, sep=' ')
    val_list = pd.read_csv(os.path.join(root_path, 'val_list_rawframes.txt'), header=None, sep=' ')
    test_list = pd.read_csv(os.path.join(root_path, 'test_list_rawframes.txt'), header=None, sep=' ')
    label_map = pd.read_csv(os.path.join(root_path, 'label_map.txt'), header=None)
    label_map = [i[0] for i in label_map.values]
    for file_list in [train_list, val_list, test_list]:
        file_list.columns = ['dir_name', 'num_frames', 'label']
    return train_list, val_list, test_list, label_map

In [3]:
def save_imgs(files_list, phase=None):
    global root_path, dst_path, label_str, label_count
    dir_names = files_list[files_list['label'] == label_idx]['dir_name'].unique()
    if phase == 'train':
        rawframe_dirname = 'rawframes_train'
    elif phase == 'val':
        rawframe_dirname = 'rawframes_val'
    elif phase == 'test':
        rawframe_dirname = 'rawframes_test'
    dir_label_path = os.path.join(dst_path, rawframe_dirname.replace('rawframes', 'otecls'), label_str)
    if not os.path.exists(dir_label_path):
        os.makedirs(dir_label_path)
    for dir_name in dir_names:
        dir_imgs = glob.glob(os.path.join(root_path, rawframe_dirname, str(dir_name), '*'))
        for dir_img in dir_imgs:
            img_name = dir_img.split('/')[-1]
            img_idx = format(label_count, '>06')
            shutil.copy(dir_img, os.path.join(dst_path, rawframe_dirname.replace('rawframes', 'otecls'), label_str, img_idx+'.jpg'))
            label_count += 1

In [4]:
root_path = 'jester/SC_jester_5cls_12_samples_seed_5'
dst_path = 'otecls_jester_5cls_12_samples_seed_5'
if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
train_list, val_list, test_list, label_map = parse_metainfo(root_path)

num_labels = len(label_map)
for label_idx in range(num_labels):
    label_count = 0
    label_str = label_map[label_idx].replace(' ', '_')
    save_imgs(train_list, phase='train')
    save_imgs(val_list, phase='val')
    save_imgs(test_list, phase='test')